In [ ]:
# Install necessary libraries
!pip install numpy pandas tensorflow music21 gradio

# Import libraries
import numpy as np
import music21
import glob
import pickle
from google.colab import files
from collections import Counter
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Step 1: Upload MIDI files
uploaded = files.upload()

# Step 2: Check uploaded files
import os
print("Uploaded files:", os.listdir())

# Step 3: Load MIDI files
midi_files = glob.glob('*.mid')  # Adjusted to match uploaded MIDI files
print("MIDI files found:", midi_files)

# Step 4: Helper function to parse MIDI files
def parse_midi(file):
    notes = []
    try:
        midi = music21.converter.parse(file)
        parts = music21.instrument.partitionByInstrument(midi)
        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    except Exception as e:
        print(f"Error parsing {file}: {e}")

    print(f"Notes parsed from {file}: {len(notes)}")
    return notes

# Step 5: Parse all MIDI files
all_notes = []
for file in midi_files:
    notes = parse_midi(file)
    all_notes.extend(notes)

# Check if notes were extracted
if len(all_notes) == 0:
    print("No notes were extracted. Please check your MIDI files.")
else:
    print("Total notes extracted:", len(all_notes))

# Step 6: Save all notes to a file for future use
with open('all_notes.pkl', 'wb') as f:
    pickle.dump(all_notes, f)

# Step 7: Load saved notes
with open('all_notes.pkl', 'rb') as f:
    all_notes = pickle.load(f)

# Step 8: Unique notes and mapping to integers
unique_notes = list(set(all_notes))
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# Step 9: Prepare input sequences and corresponding outputs
SEQUENCE_LENGTH = 100
network_input = []
network_output = []

for i in range(0, len(all_notes) - SEQUENCE_LENGTH):
    sequence_in = all_notes[i:i + SEQUENCE_LENGTH]
    sequence_out = all_notes[i + SEQUENCE_LENGTH]
    network_input.append([note_to_int[note] for note in sequence_in])
    network_output.append(note_to_int[sequence_out])

# Reshape and normalize input
if network_input and network_output:  # Check if there are valid inputs
    network_input = np.reshape(network_input, (len(network_input), SEQUENCE_LENGTH, 1))
    network_input = network_input / len(unique_notes)  # Normalize input
    network_output = to_categorical(network_output)

    # Step 10: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(network_input, network_output, test_size=0.2)
else:
    print("Error: network_input or network_output is empty. Check your MIDI files and parsing.")


In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

model = Sequential()
model.add(LSTM(512, input_shape=(SEQUENCE_LENGTH, 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(len(unique_notes)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=2, batch_size=64, validation_data=(X_test, y_test))
model.save('music_model.h5')


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                        │ (None, 100, 512)            │       1,052,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 512)                 │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 100)                 │          25,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 100)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,308,900 (12.62 MB)

 Trainable params: 3,308,900 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
41/41 ━━━━━━━━━━━━━━━━━━━━ 223s 5s/step - loss: 4.2814 - val_loss: 3.9244
Epoch 2/2
41/41 ━━━━━━━━━━━━━━━━━━━━ 263s 5s/step - loss: 3.9678 - val_loss: 3.8963


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('music_model.h5')

def generate_music(model, start_sequence, num_notes):
    int_to_note = {number: note for note, number in note_to_int.items()}
    generated_notes = start_sequence

    for i in range(num_notes):
        input_sequence = np.reshape(generated_notes[-SEQUENCE_LENGTH:], (1, SEQUENCE_LENGTH, 1))
        input_sequence = input_sequence / len(unique_notes)
        prediction = model.predict(input_sequence, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        generated_notes.append(index)

    return [int_to_note[i] for i in generated_notes]

# Starting sequence
start_sequence = [note_to_int[note] for note in all_notes[:SEQUENCE_LENGTH]]
generated_notes = generate_music(model, start_sequence, 500)

# Convert generated notes to MIDI
output_notes = []
for pattern in generated_notes:
    if ('.' in pattern) or pattern.isdigit():
        chord_notes = pattern.split('.')
        notes = [music21.note.Note(int(n)) for n in chord_notes]
        chord = music21.chord.Chord(notes)
        output_notes.append(chord)
    else:
        note = music21.note.Note(pattern)
        output_notes.append(note)

midi_stream = music21.stream.Stream(output_notes)
midi_stream.write('midi', fp='generated_music.mid')


In [ ]:
import gradio as gr

def generate_music_interface(num_notes):
    generated_notes = generate_music(model, start_sequence, num_notes)
    output_notes = []
    for pattern in generated_notes:
        if ('.' in pattern) or pattern.isdigit():
            chord_notes = pattern.split('.')
            notes = [music21.note.Note(int(n)) for n in chord_notes]
            chord = music21.chord.Chord(notes)
            output_notes.append(chord)
        else:
            note = music21.note.Note(pattern)
            output_notes.append(note)

    midi_stream = music21.stream.Stream(output_notes)
    midi_stream.write('midi', fp='generated_music_output.mid')

    return 'generated_music_output.mid'

interface = gr.Interface(fn=generate_music_interface, inputs="slider", outputs="file", description="Generate AI Music")
interface.launch()
